# Shear profile around A360 using ComCam HSM shapes

Author: Céline Combet (with input from many - Anthony Englert, Shenming Fu, Ian dell'Antonio, Pakruth Adari,... see [SITCOMTN-161](https://sitcomtn-161.lsst.io/))\
LSST Science Piplines version: Weekly 2025_17\
Container Size: medium

This notebook provides the code used to generate the shear profile figure in section 4 of [SITCOMTN-161](https://sitcomtn-161.lsst.io/) (it acutally also produces other figures that were not shown in the TechNote). The main steps are:

- Loading the relevant object catalogs (all tracts and patches needed) using the butler
- Color cut source selection
- HSC lensing quality cuts
- HSC Y1 calibration step. The `gen_hsc_calibration` script needs to be installed. The script is publicly available at: [https://github.com/PrincetonUniversity/hsc-y1-shear-calib](https://github.com/PrincetonUniversity/hsc-y1-shear-calib)
- Shear profile measurement using [CLMM](https://github.com/LSSTDESC/clmm). In the RSP, using a LSST kernel, you should have CLMM installed locally (requires `qp` and `pyccl` installed as well) prior to running the last steps.


In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re


In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

In [ ]:
repo = '/repo/dp1'
collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

butler = Butler(repo, collections=collection)

In [ ]:
version_str = collection.split('/')
version = version_str[-2:][0]+'_'+version_str[-2:][1]
version

In [ ]:
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

## Find tracts and patches for Abell 360 and load the catalogs

Find all the tracts/patches that falls in a given region around the A360 BCG, and store the results in a dictionary `tp_dict`

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict
#print(tracts_and_patches)

Load the object catalogs for all these tracts/patches, make basic cuts, and store in a single merged catalog `merged_cat`.

In [ ]:
# Get the object catlaog of these patches
if 'v29' in version:
    datasetType = 'object_patch'
else:
    datasetType = 'objectTable'
 
merged_cat = pd.DataFrame()

for tract in tp_dict.keys():
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')
    for patch in tp_dict[tract]:
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get(datasetType, dataId=dataId)
        if datasetType == 'object_patch': # new naming convention, and obj_cat is now an astropy table. 
            obj_cat = obj_cat.to_pandas() # convert to pandas to leave the rest of the code unchanged
        pattern = r"^g_|^z_"  
        # Drop columns matching the pattern
        obj_cat = obj_cat.drop(columns=[col for col in obj_cat.columns if re.match(pattern, col)])

        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['r_cModel_flag']== False
        filt &= obj_cat['i_cModel_flag']== False
        filt &= obj_cat['r_cModelFlux']>0
        filt &= obj_cat['i_cModelFlux']>0
        filt &= obj_cat['refExtendedness'] > 0.5


        merged_cat = pd.concat([merged_cat, obj_cat[filt]], ignore_index=True)
        

## Red sequence identification

### Select a circular field close (<0.1 deg) to the BCG in order to identify RS

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

c1 = SkyCoord(merged_cat['coord_ra'].values*u.deg, merged_cat['coord_dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

sep.deg
filt = sep.deg < 0.1 # stay close to cluster center for RS indentification
merged_cat_rs = merged_cat[filt] # catalog for RS identification

In [ ]:
len(merged_cat), len(merged_cat_rs)

### Convert fluxes to magnitudes and identify red sequence in r-i versus r

Conversion from fluxes to mag using formula from DP0.2 tutorial.

In [ ]:
mag_i = -2.50 * np.log10(merged_cat_rs['i_cModelFlux']) + 31.4
mag_r = -2.50 * np.log10(merged_cat_rs['r_cModelFlux']) + 31.4

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
ax[0].hist(mag_r, bins=50)
ax[1].hist(mag_i, bins=50)
ax[0].set_yscale('log')
ax[1].set_yscale('log')

### Color magnitude diagram and by eye indetification of the red sequence


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,4))
ax.scatter(mag_r, mag_r-mag_i, marker='.', s=0.3)
ax.set_ylim([-2,2])
ax.set_xlim([19,25])
ax.set_ylabel('r-i')
ax.set_xlabel('r')
ax.plot([19,24],[0.44,0.34], color='r', linewidth=0.7)
ax.plot([19,24],[0.64,0.54], color='r', linewidth=0.7)
fig.tight_layout()
# fig.savefig(f"RS_{version}.png")

### Filter to identify red sequence galaxies in the sample


In [ ]:
rs_hi = 0.64 - (0.1/5.) * (mag_r-19)
rs_low = 0.44 - (0.1/5.)* (mag_r-19)
color = mag_r - mag_i

idx = np.where(np.logical_and(color>rs_low, color<rs_hi))[0]
idx2 = np.where(mag_r.iloc[idx] < 22)[0] # keep the brightest objects only

RS_id_list_nearcluster = merged_cat_rs['objectId'].iloc[idx].iloc[idx2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, 
                         figsize=(4,4))
ax.scatter(mag_r, mag_r-mag_i, marker='.', s=0.3) # all galaxies  
ax.scatter(mag_r.iloc[idx].iloc[idx2], 
           mag_r.iloc[idx].iloc[idx2]-mag_i.iloc[idx].iloc[idx2], 
           marker='.', s=0.3) #red sequence galaxies
ax.set_ylim([-2,2])
ax.set_xlim([19,27])
ax.set_ylabel('r-i')
ax.set_xlabel('r')
ax.plot([19,24],[0.44,0.34], color='r', linewidth=0.7)
ax.plot([19,24],[0.64,0.54], color='r', linewidth=0.7)

## Remove red sequence galaxies in the full field

For the analysis, we'll keep source galaxies within 0.5 deg from the BCG. Now we apply the RS cut defined on the small region above to the full field of the analysis. The RS-free catalog is stored as `merged_cat_wl`. The lensing quality cuts will be performed in a subsequent step. 

In [ ]:
filt = sep.deg <  10 #0.5 # larger field for analysis
merged_cat_wl = merged_cat[filt]

In [ ]:
mag_i = -2.50 * np.log10(merged_cat_wl['i_cModelFlux']) + 31.4
mag_r = -2.50 * np.log10(merged_cat_wl['r_cModelFlux']) + 31.4
color = mag_r - mag_i

# Filter defined above applied to the full sample
rs_hi = 0.64 - (0.1/5.) * (mag_r-19)
rs_low = 0.44 - (0.1/5.)* (mag_r-19)

idx = np.where(np.logical_and(color>rs_low, color<rs_hi))[0]
idx2 = np.where(mag_r.iloc[idx] < 22)[0] # keep the brightest objects only

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, 
                         figsize=(4,4))
ax.scatter(mag_r, mag_r-mag_i, marker='.', s=0.3) # all galaxies  
ax.scatter(mag_r.iloc[idx].iloc[idx2], 
           mag_r.iloc[idx].iloc[idx2]-mag_i.iloc[idx].iloc[idx2], 
           marker='.', s=0.3) #red sequence galaxies
ax.set_ylim([-2,2])
ax.set_xlim([19,27])
ax.set_ylabel('r-i')
ax.set_xlabel('r')

In [ ]:
RS_id_list = merged_cat_wl['objectId'].iloc[idx].iloc[idx2]

In [ ]:
# Filter out rows where the 'dataid' column matches any value in RS_id_list
merged_cat_wl = merged_cat_wl[~merged_cat_wl['objectId'].isin(RS_id_list)]

## Lensing cuts

The RS sequence has been removed. Now apply a series of lensing cuts (mostly following Shenming's [CLMM HSC demo analysis](https://github.com/LSSTDESC/CLMM/blob/main/examples/mass_fitting/Example4_Fit_Halo_mass_to_HSC_data.ipynb), but missing some at the moment), to the `merged_cat_wl` catalog. There might be more cuts to implement to improve sample purity.

In [ ]:
len(merged_cat_wl)

In [ ]:
# Compute again magnitudes, but for the RS-free catalog
mag_i = -2.50 * np.log10(merged_cat_wl['i_cModelFlux']) + 31.4
mag_r = -2.50 * np.log10(merged_cat_wl['r_cModelFlux']) + 31.4

# Filters to keep sources with good-quality measured shape in i band
source_filt = np.sqrt(merged_cat_wl['i_hsmShapeRegauss_e1']**2 + merged_cat_wl['i_hsmShapeRegauss_e2']**2) < 4
source_filt &= merged_cat_wl['i_hsmShapeRegauss_sigma']<= 0.4
source_filt &= merged_cat_wl['i_hsmShapeRegauss_flag'] == 0
source_filt &= merged_cat_wl['i_blendedness'] < 0.42
source_filt &= merged_cat_wl['i_iPSF_flag']==0

# Resolution factor quality cut - according to Mandelbaum (2018) paper:
# "we use the resolution factor R2 which is defined using the traces of the moment matrix of the PSF TP and 
# of the observed (PSF-convolved) galaxy image TI as: R2 = 1- TP/TI"
# Best guess to translate that in terms of ComCam objectTable catalog output...

res = 1 - (merged_cat_wl['i_ixxPSF']+ merged_cat_wl['i_iyyPSF']) / (merged_cat_wl['i_ixx']+ merged_cat_wl['i_iyy'])
source_filt &= res >= 0.3

source_filt &= (mag_i <= 24.5) & (mag_i>21) 

print(f'Source sample size: {np.sum(source_filt)}')

### Final source sample CMD, (ra,dec) distribution, etc.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, 
                         figsize=(4,4))
ax.scatter(mag_r[source_filt], mag_r[source_filt]-mag_i[source_filt], marker='.', s=0.3) # all galaxies  
ax.set_ylim([-1,2])
ax.set_xlim([18,27])
ax.set_ylabel('r-i')
ax.set_xlabel('r')

In [ ]:
ra = merged_cat_wl['coord_ra'][source_filt]
dec = merged_cat_wl['coord_dec'][source_filt]
e1 = merged_cat_wl['i_hsmShapeRegauss_e1'][source_filt]
e2 = merged_cat_wl['i_hsmShapeRegauss_e2'][source_filt]
# e_err = merged_cat_wl['i_hsmShapeRegauss_sigma'][source_filt]
gal_id = merged_cat_wl['objectId'][source_filt]

In [ ]:
plt.scatter(ra, dec, marker='.', s=0.2)
plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')

## Apply HSC shear calibration

### Save source catalog `merged_cat_wl` as fits file to use as input for the HSC calibration script.

In [ ]:
from astropy.io import fits
from astropy.table import Table, vstack
import pandas as pd

# Too many columns in the pandas dataframe. Remove some unecessary ones.
import re
# Define pattern (example: drop all columns starting with "temp_")
pattern = r"^g_|^z_"  
# Drop columns matching the pattern
merged_cat_wl = merged_cat_wl.drop(columns=[col for col in merged_cat_wl.columns if re.match(pattern, col)])

astropy_table = Table.from_pandas(merged_cat_wl[source_filt])
astropy_table.write('source_sample.fits', format="fits", overwrite=True)

Now that the source_sample.fits file exists, need to use the HSC calibration. The `get_snr, get_res, get_psf_ellip` functions in the `utilities.py` file from the HSC calibration repo first need to be updated to use the column names of DP1. Then syntax is:
```
python gen_hsc_calibrations.py source_sample.fits source_sample_calib.fits
```
which will create the `source_sample_calib.fits` file that is read below.

In [ ]:
!rm source_sample_calib.fits
!cd ../../my_softs/hsc-y1-shear-calib/ && python gen_hsc_calibrations.py ../../ComCam/comcam_clusters/source_sample.fits ../../ComCam/comcam_clusters/source_sample_calib.fits

### Read in the calibration quantities and apply the calibration

In [ ]:
from astropy.io import fits
from astropy.table import Table, vstack
import pandas as pd

with fits.open('source_sample_calib.fits') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    table = Table(data)

sigma_e = table['ishape_hsm_regauss_derived_sigma_e']
e_rms = table["ishape_hsm_regauss_derived_rms_e"]
m = table["ishape_hsm_regauss_derived_shear_bias_m"]
c1 = table["ishape_hsm_regauss_derived_shear_bias_c1"]
c2 = table["ishape_hsm_regauss_derived_shear_bias_c2"]
weight = table["ishape_hsm_regauss_derived_shape_weight"]

to_use = np.isfinite(weight)*np.isfinite(e_rms)*np.isfinite(m)*np.isfinite(c1)*np.isfinite(c2)
e1_0 = e1[to_use]
e2_0 = e2[to_use]
e_rms = e_rms[to_use]
c1 = c1[to_use]
c2 = c2[to_use]
m = m[to_use]
weight = weight[to_use]
e_err = sigma_e[to_use]

print(f'Number of sources with calibration: {np.sum(to_use)}')

In [ ]:
# From Shenming's CLMM demo on using HSC data
def apply_shear_calibration(e1_0, e2_0, e_rms, m, c1, c2, weight):
    R = 1.0 - np.sum(weight * e_rms**2.0) / np.sum(weight)
    m_mean = np.sum(weight * m) / np.sum(weight)
    c1_mean = np.sum(weight * c1) / np.sum(weight)
    c2_mean = np.sum(weight * c2) / np.sum(weight)
    print("R, m_mean, c1_mean, c2_mean: ", R, m_mean, c1_mean, c2_mean)

    g1 = (e1_0 / (2.0 * R) - c1) / (1.0 + m_mean)
    g2 = (e2_0 / (2.0 * R) - c2) / (1.0 + m_mean)

    return g1, g2

In [ ]:
g1, g2 = apply_shear_calibration(e1_0, e2_0, e_rms, m, c1, c2, weight)

In [ ]:
plt.hist(e1[to_use], bins=100, alpha=0.2, range=[-2, 2], label='e1');
plt.hist(g1, bins=100, alpha=0.2,range=[-2, 2], label='g1 - HSC calibration');
plt.legend()

## Use calibrated shears with CLMM to measure the shear profile

In [ ]:
import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

Prepare a CLMM GCData table using the catalog

In [ ]:
galcat = GCData()
galcat['id'] = gal_id[to_use]
galcat['ra'] = ra[to_use]
galcat['dec'] = dec[to_use]
# galcat['e1'] = e1[to_use]
# galcat['e2'] = e2[to_use]
galcat['e1'] = g1
galcat['e2'] = g2
#galcat['e_err'] = e_err[to_use]/2.  # factor 2 to account for conversion between e and g
galcat['e_err'] = e_err/2.  # factor 2 to account for conversion between e and g

galcat['z'] = np.zeros(len(ra[to_use])) # CLMM needs a redshift column for the source, even if not used

Create the corresponding CLMM galaxy cluster object

In [ ]:
cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')

In [ ]:
gc_object1.compute_tangential_and_cross_components(add=True);

Compute the lensing weights using CLMM - to do: to be checked against the weights that come out of the calibration step.


In [ ]:
gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=True,
        shape_component1_err="e_err",
        shape_component2_err="e_err",
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    ) 
#gc_object1.galcat['w_ls']=weight #use the weights from the HSC calibration, and not the CLMM-computed ones

In [ ]:
gc_object1.galcat[0:5]

In [ ]:
# Radial binning, either in Mpc or degrees
bins_mpc = clmm.make_bins(0.4,5,nbins=6, method='evenlog10width')

Radial profile computation

In [ ]:
gc_object1.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=True,gal_ids_in_bins=True, error_model='ste');

In [ ]:
# Check the profile table
#gc_object1.profile[0]

Also use CLMM to get a typical model for a cluster at that redshift, assuming the DESC SRD n(z)

In [ ]:
moo = clmm.Modeling(massdef="critical", delta_mdef=500, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(4.e14)

z_cl = gc_object1.z

# source properties
# Wrongly assume sources redshift following a the DESC SRD distribution (pre-coded in CLMM). 
# Just to get something in the ballpark of what to expect

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.1),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
plt.errorbar(gc_object1.profile['radius'], gc_object1.profile['gt'], gc_object1.profile['gt_err'], 
             ls='', marker='.', label='tangential')
plt.errorbar(gc_object1.profile['radius']*1.02, gc_object1.profile['gx'], gc_object1.profile['gx_err'], 
             ls='', marker='.', label='cross')
plt.plot(rproj, gt_z, label='NFW, M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.06,0.1])
plt.xlim([0.4,6])
#plt.xlim([0.0,0.6])
#plt.yscale('log')
plt.xlabel('R [Mpc]')
#plt.xlabel('separation [deg]')
plt.ylabel('reduced shear')
plt.legend(loc=1)
plt.savefig(f"profile_{version}.png")
plt.savefig('image_output/shear_profile.png')